In [12]:
import pandas as pd
import numpy as np


In [17]:
df = pd.read_csv("C:\\Project\\corona_tested_individuals_ver_00225.csv")
df

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,2022-02-17,0,0,0,0,0,חיובי,No,נקבה,Other
1,2022-02-17,0,0,0,0,0,שלילי,No,נקבה,Other
2,2022-02-17,0,0,0,0,0,שלילי,No,נקבה,Other
3,2022-02-17,0,0,0,0,0,שלילי,No,NaN,Other
4,2022-02-17,0,0,0,0,0,שלילי,No,NaN,Other
...,...,...,...,...,...,...,...,...,...,...
9053443,2020-03-11,0,1,0,1,0,שלילי,NaN,NaN,Abroad
9053444,2020-03-11,0,0,0,0,0,שלילי,NaN,NaN,Other
9053445,2020-03-11,0,0,0,0,0,שלילי,NaN,NaN,Other
9053446,2020-03-11,0,0,0,0,0,שלילי,NaN,NaN,Other


In [18]:
df.isnull().sum()
df = df.dropna()

In [19]:
df

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,2022-02-17,0,0,0,0,0,חיובי,No,נקבה,Other
1,2022-02-17,0,0,0,0,0,שלילי,No,נקבה,Other
2,2022-02-17,0,0,0,0,0,שלילי,No,נקבה,Other
7,2022-02-17,0,0,0,0,0,שלילי,No,נקבה,Other
8,2022-02-17,0,0,0,0,0,שלילי,Yes,נקבה,Other
...,...,...,...,...,...,...,...,...,...,...
9038852,2020-03-22,0,0,0,0,0,שלילי,No,זכר,Other
9038853,2020-03-22,1,0,0,0,0,שלילי,No,נקבה,Abroad
9038854,2020-03-22,1,1,0,0,0,חיובי,No,זכר,Other
9038855,2020-03-22,0,0,0,0,0,שלילי,No,זכר,Other


In [20]:
df.head()

,test_date,cough,fever,sore_throat,shortness_of_breath,head_ache,corona_result,age_60_and_above,gender,test_indication
0,2022-02-17,0,0,0,0,0,חיובי,No,נקבה,Other
1,2022-02-17,0,0,0,0,0,שלילי,No,נקבה,Other
2,2022-02-17,0,0,0,0,0,שלילי,No,נקבה,Other
7,2022-02-17,0,0,0,0,0,שלילי,No,נקבה,Other
8,2022-02-17,0,0,0,0,0,שלילי,Yes,נקבה,Other


In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6344272 entries, 0 to 9038856
Data columns (total 10 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   test_date            object
 1   cough                int64 
 2   fever                int64 
 3   sore_throat          int64 
 4   shortness_of_breath  int64 
 5   head_ache            int64 
 6   corona_result        object
 7   age_60_and_above     object
 8   gender               object
 9   test_indication      object
dtypes: int64(5), object(5)
memory usage: 532.4+ MB


In [31]:
df['gender'] = df['gender'].replace({"זכר": 1, "נקבה": 0})
df['corona_result'] = df['corona_result'].replace({"חיובי": 1, "שלילי": 0})
df['age_60_and_above'] = df['age_60_and_above'].replace({"Yes": 1, "No": 0})


In [40]:
df.describe()

,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender
count,6.344272e+06,6.344272e+06,6.344272e+06,6.344272e+06,6.344272e+06,6.344272e+06,6.344272e+06
mean,5.020497e-02,3.895561e-02,2.136400e-02,4.303409e-03,4.154551e-02,1.303629e-01,4.853337e-01
std,2.183677e-01,1.934892e-01,1.445945e-01,6.545908e-02,1.995482e-01,3.367023e-01,4.997849e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [39]:
df[df['corona_result']==1].describe()


,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender
count,826347.000000,826347.000000,826347.000000,826347.000000,826347.000000,826347.000000,826347.000000
mean,0.138974,0.116986,0.064633,0.016017,0.125158,0.127350,0.483819
std,0.345920,0.321404,0.245877,0.125543,0.330898,0.333364,0.499738
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [41]:
df[df['corona_result']==0].describe()

,cough,fever,sore_throat,shortness_of_breath,head_ache,age_60_and_above,gender
count,5.381036e+06,5.381036e+06,5.381036e+06,5.381036e+06,5.381036e+06,5.381036e+06,5.381036e+06
mean,3.663960e-02,2.707100e-02,1.474344e-02,2.497474e-03,2.878702e-02,1.308841e-01,4.854719e-01
std,1.878753e-01,1.622904e-01,1.205242e-01,4.991230e-02,1.672075e-01,3.372736e-01,4.997889e-01
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
max,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00


In [37]:
df['gender'].value_counts()

0    3265183
1    3079089
Name: gender, dtype: int64